This notebook contains the code samples found in Chapter 2, Section 1 of
[Deep Learning with R](https://www.manning.com/books/deep-learning-with-r). Note
that the original text features far more content, in particular further
explanations and figures: in this notebook, you will only find source code and
related comments.


In [2]:
Sys.setenv("RETICULATE_PYTHON"="C:/Users/EQUIPO/.ai-navigator/conda/envs/OHW")

library(keras)
library(tensorflow)

Registered S3 methods overwritten by 'keras':
  method                               from  
  as.data.frame.keras_training_history keras3
  plot.keras_training_history          keras3
  print.keras_training_history         keras3
  r_to_py.R6ClassGenerator             keras3



In [3]:
tf_config()

TensorFlow v2.17.0 (C:\Users\EQUIPO\AI-NAV~1\conda\envs\OHW\lib\site-packages\keras\__init__.p)
Python v3.9 (C:/Users/EQUIPO/.ai-navigator/conda/envs/OHW/python.exe)

In [4]:
tf_version()

[1] '2.17'

Let's look at a concrete example of a neural network that uses the Keras R
package to learn to classify hand-written digits. Unless you already have
experience with Keras or similar libraries, you will not understand everything
about this first example right away. You probably haven't even installed Keras
yet. Don't worry, that is perfectly fine. In the next chapter, we will review
each element in our example and explain them in detail. So don't worry if some
steps seem arbitrary or look like magic to you! We've got to start somewhere.

The problem we're trying to solve here is to classify grayscale images of
handwritten digits (28 pixels by 28 pixels) into their 10 categories (0 to 9).
We'll use the MNIST dataset, a classic dataset in the machine-learning
community, which has been around almost as long as the field itself and has been
intensively studied. It's a set of 60,000 training images, plus 10,000 test
images, assembled by the National Institute of Standards and Technology (the
NIST in MNIST) in the 1980s. You can think of "solving" MNIST as the "Hello
World" of deep learning—it's what you do to verify that your algorithms are
working as expected. As you become a machine-learning practitioner, you'll see
MNIST come up over and over again, in scientific papers, blog posts, and so on.

The MNIST dataset comes preloaded in Keras, in the form of `train` and `test`
lists, each of which includes a set of images (`x`) and associated labels (`y`):


In [5]:
mnist <- dataset_mnist()
train_images <- mnist$train$x
train_labels <- mnist$train$y
test_images <- mnist$test$x
test_labels <- mnist$test$y

`train_images` and `train_labels` form the _training set_, the data that the
model will learn from. The model will then be tested on the _test set_,
`test_images` and `test_labels`. The images are encoded as as 3D arrays, and the
labels are a 1D array of digits, ranging from 0 to 9. There is a one-to-one
correspondence between the images and the labels.

The R `str()` function is a convenient way to get a quick glimpse at the
structure of an array. Let's use it to have a look at the training data:


In [6]:
str(train_images)

 int [1:60000, 1:28, 1:28] 0 0 0 0 0 0 0 0 0 0 ...


In [7]:
train_images[1,,]

0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,175,26,166,255,247,127,0,0,0,0
0,0,0,0,0,0,0,0,30,36,...,225,172,253,242,195,64,0,0,0,0
0,0,0,0,0,0,0,49,238,253,...,93,82,82,56,39,0,0,0,0,0
0,0,0,0,0,0,0,18,219,253,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,80,156,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,14,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train_labels[1]

[1] 5

In [9]:
str(train_labels)

 int [1:60000(1d)] 5 0 4 1 9 2 1 3 1 4 ...


In [10]:
str(test_images)

 int [1:10000, 1:28, 1:28] 0 0 0 0 0 0 0 0 0 0 ...


The workflow will be as follows: first we'll feed the neural network the
training data, `train_images` and `train_labels`. The network will then learn to
associate images and labels. Finally, we'll ask the network to produce
predictions for `test_images`, and we'll verify whether these predictions match
the labels from `test_labels`.

Let's build the network -- again, remember that you aren't supposed to
understand everything about this example yet.


In [11]:
network <- keras_model_sequential() %>% 
  layer_dense(units = 512, 
              activation = "relu", 
              input_shape = c(28 * 28)) %>% 
  layer_dense(units = 10, 
              activation = "softmax")

The core building block of neural networks is the _layer_, a data-processing
module that you can think of as a filter for data. Some data comes in, and it
comes out in a more useful form. Specifically, layers extract _representations_
out of the data fed into them—hopefully representations that are more meaningful
for the problem at hand. Most of deep learning consists of chaining together
simple layers that will implement a form of progressive _data distillation_. A
deep-learning model is like a sieve for data processing, made of a succession of
increasingly refined data filters—the layers.

Here our network consists of a sequence of two layers, which are densely
connected (also called _fully connected_) neural layers. The second (and last)
layer is a 10-way _softmax_ layer, which means it will return an array of 10
probability scores (summing to 1). Each score will be the probability that the
current digit image belongs to one of our 10 digit classes.

To make the network ready for training, we need to pick three more things, as
part of the _compilation_ step:

- _A loss function_—How the network will be able to measure how good a job it's
  doing on its training data, and thus how it will be able to steer itself in
  the right direction.
- _An optimizer_—The mechanism through which the network will update itself
  based on the data it sees and its loss function.
- _Metrics to monitor during training and testing_—Here we'll only care about
  accuracy (the fraction of the images that were correctly classified).

The exact purpose of the loss function and the optimizer will be made clear
throughout the next two chapters.


In [12]:
network %>% compile(
  optimizer = "rmsprop",
  loss = "categorical_crossentropy",
  metrics = c("accuracy")
)

Before training, we'll preprocess the data by reshaping it into the shape the
network expects and scaling it so that all values are in the `[0, 1]` interval.
Previously, our training images, for instance, were stored in an array of shape
`(60000, 28, 28)` of type integer with values in the `[0, 255]` interval. We
transform it into a double array of shape `(60000, 28 * 28)` with values between
0 and 1.


In [14]:
train_images < -array_reshape(train_images, c(60000, 28 * 28))
train_images < -train_images / 255

test_images < -array_reshape(test_images, c(10000, 28 * 28))
test_images < -test_images / 255

ERROR: Error in train_images < -array_reshape(train_images, c(60000, 28 * 28)): arreglos de dimensón no compatibles


We also need to categorically encode the labels, a step which we explain in
chapter 3:


In [15]:
train_labels < -to_categorical(train_labels)
test_labels < -to_categorical(test_labels)

ERROR: Error in (structure(function (x, num_classes = NULL) : unused arguments (y = c(5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1, 1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9, 3, 9, 8, 5, 9, 3, 3, 0, 7, 4, 9, 8, 0, 9, 4, 1, 4, 4, 6, 0, 4, 5, 6, 1, 0, 0, 1, 7, 1, 6, 3, 0, 2, 1, 1, 7, 9, 0, 2, 6, 7, 8, 3, 9, 0, 4, 6, 7, 4, 6, 8, 0, 7, 8, 3, 1, 5, 7, 1, 7, 1, 1, 6, 3, 0, 2, 9, 3, 1, 1, 0, 4, 9, 2, 0, 0, 2, 0, 2, 7, 1, 8, 6, 4, 1, 6, 3, 4, 5, 9, 1, 3, 3, 8, 5, 4, 7, 7, 4, 2, 8, 5, 8, 6, 7, 3, 4, 6, 1, 9, 9, 6, 0, 3, 7, 2, 8, 2, 
9, 4, 4, 6, 4, 9, 7, 0, 9, 2, 9, 5, 1, 5, 9, 1, 2, 3, 2, 3, 5, 9, 1, 7, 6, 2, 8, 2, 2, 5, 0, 7, 4, 9, 7, 8, 3, 2, 1, 1, 8, 3, 6, 1, 0, 3, 1, 0, 0, 1, 7, 2, 7, 3, 0, 4, 6, 5, 2, 6, 4, 7, 1, 8, 9, 9, 3, 0, 7, 1, 0, 2, 0, 3, 5, 4, 6, 5, 8, 6, 3, 7, 5, 8, 0, 9, 1, 0, 3, 1, 2, 2, 3, 3, 6, 4, 7, 5, 0, 6, 2, 7, 9, 8, 5, 9, 2, 1, 1, 4, 4, 5, 6, 4, 1, 2, 5, 3, 9, 3, 9, 0, 5, 9, 6, 5, 7, 4, 1, 3, 4, 0, 4, 8, 0, 4, 3, 6, 8, 7, 6, 0, 9

We are now ready to train our network, which in Keras is done via a call to the
`fit` method of the network: we "fit" the model to its training data.


In [ ]:
network %>% fit(train_images, 
                train_labels, 
                epochs = 5, 
                batch_size = 128)

Two quantities are being displayed during training: the "loss" of the network
over the training data, and the accuracy of the network over the training data.

We quickly reach an accuracy of 0.989 (i.e. 98.9%) on the training data. Now
let's check that our model performs well on the test set too:


In [ ]:
metrics <- network %>% evaluate(test_images, test_labels, verbose = 0)
metrics